# URL

In [ ]:
# import urllib.request
# url = 'http://cirtec.ranepa.ru/analysis/Word2Vec/citcon4bundles.txt'
# response = urllib.request.urlopen(url)
# data = response.read()      # a `bytes` object

# with open('../initial_data/Word2Vec__citcon4bundles.txt', 'wb') as f:
#     f.write(data)

# FILE

Загрузка файл, подготовка данных в виде:
1) массива слов
2) массива нормализованных слов

~10 минут

In [ ]:
%%time
import string
import re
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
regex = re.compile('[^а-яА-Я]')

def get_average_len_words(sentence):
    words = sentence.split()
    if len(words) == 0:
        return 0
    return sum([len(word) for word in words]) / len(words)

with open('../initial_data/Word2Vec__citcon4bundles.txt', 'rb') as f:
    data = f.read()
text = data.decode('utf-8') # a `str`; this step can't be used if data is binary
del data
sentences = [(sentence[0], sentence[1], sentence[2], sentence[3], sentence[4].replace("\xad ", "").lower()) 
             for sentence in (sentence.split(" ", 4) for sentence in text.split("\n") if sentence and get_average_len_words(sentence) > 5)]
del text
print("len_sentences: {}".format(len(sentences)))

words = [(sentence[0], [word for word in regex.sub(' ', sentence[4]).split()]) for sentence in sentences]
words_normal_form = [(sentence[0], [morph.parse(word)[0].normal_form for word in regex.sub(' ', sentence[4]).split()]) for sentence in sentences]

del sentences

In [ ]:
from gensim.models.phrases import Phrases

def get_n_gram(_words, n, delimiters=(b"@", b"#", b"$", b"%"), min_count=20, threshold=50):
    if n < 2:
        raise ValueError(" n < 2 ")
    if len(delimiters) < n - 1:
        raise ValueError(" len(delimiters) < n-1 ")
    grams = []
    for ind in range(n - 1):
        gram = Phrases(_words, min_count=min_count, delimiter=delimiters[ind], threshold=threshold)
        grams.append(gram)
        if ind != n - 2:
            _words = gram[_words]
    return grams


def drop_all_delimiters(text, delimiters=("@", "#", "$", "%")):
    for delimiter in delimiters:
        text = text.replace(delimiter, " ")
    return text


def get_gram_vocab(grams, n, delimiters=("@", "#", "$", "%")):
    if n < 2:
        raise ValueError(" n < 2 ")
    if len(delimiters) < n - 1:
        raise ValueError(" len(delimiters) < n-1 ")
    delimiters = delimiters[:n - 1]
    sorted_gram_vocab = sorted([(drop_all_delimiters(value, delimiters), count) for (value, count) in
                                [(value.decode('utf8'), count) for value, count in dict(grams[n - 2].vocab).items()]
                                if sum([dlm in value for dlm in delimiters]) == n - 1], key=lambda kv: kv[1],
                               reverse=True)

    return sorted_gram_vocab

Обучения фразеров

~2 минуты

In [ ]:
%%time

def get_phrases(_list_words, n, delimiters=("@", "#", "$", "%")):
    phrases = [[word for word in _words] for _words in _list_words] # _words
    for ind in range(n - 1):
        phrases = [[word for word in sent if delimiters[ind] in word] for sent in phrases]
    return phrases

grams = get_n_gram(_words=[word[1] for word in words], n=3, delimiters=(b"@", b"#", b"$", b"%"))
grams_normal_form = get_n_gram(_words=[word[1] for word in words_normal_form], n=3, delimiters=(b"@", b"#", b"$", b"%"))

t3 = grams[1][grams[0][[word[1] for word in words]]]
t3_normal_form = grams_normal_form[1][grams_normal_form[0][[word[1] for word in words_normal_form]]]

phrases_t3 = [value for value in zip([word[0] for word in words], get_phrases(t3, 3))]
phrases_t3_normal_form = [value for value in zip([word[0] for word in words_normal_form], get_phrases(t3_normal_form, 3))]

In [ ]:
from collections import Counter

bunshid = list(set([word[0] for word in words]))

def get_dict_counters_by_bunshid(__phrases, __bunshid):
    dict_res = dict()
    for un in __bunshid:
        all_ph = []
        for i in (ph for ph in __phrases if ph[0] == un):
            all_ph.extend(i[1])
        dict_res[un] = Counter(all_ph)
    return dict_res

dict_res_t3 = get_dict_counters_by_bunshid(phrases_t3, bunshid)
dict_res_t3_normal_form = get_dict_counters_by_bunshid(phrases_t3_normal_form, bunshid)

In [ ]:
# ind = 14

# print(bunshid[ind])
# print(dict_res_t3[bunshid[ind]].most_common())
# print()
# print(dict_res_t3_normal_form[bunshid[ind]].most_common())

# Create result dict

In [ ]:
result_dict = dict()
for b in bunshid:
    result_dict[b] = {
        "3orig": [{"w": drop_all_delimiters(m_c[0]), "n": m_c[1]} for m_c in dict_res_t3[b].most_common()],
        "3norm": [{"w": drop_all_delimiters(m_c[0]), "n": m_c[1]} for m_c in dict_res_t3_normal_form[b].most_common()],
    }

In [ ]:
# result_dict[bunshid[0]]

# Create json

In [ ]:
import json

with open("../resulting_data/json_grams/3_grams_v2.json", "w") as f:
    json.dump(result_dict, f)